In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.9/871.9 kB 13.2 MB/s eta 0:00:00a 0:00:01


In [32]:
from ultralytics import YOLO

## upload the dataset
> [DataSet](https://universe.roboflow.com/first-fwyb3/road-lines-segmentation-m3thn/dataset/10)

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mFrUreCV7Y9LdiwgQrx3")
project = rf.workspace("first-fwyb3").project("road-lines-segmentation-m3thn")
version = project.version(10)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 1.9 MB/s eta 0:00:00a 0:00:01
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.92, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Road-Lines-Segmentation-10 in yolov8:: 100%|██████████| 35810/35810 [00:05<00:00, 7007.35it/s]


In [18]:
TRAIN_DIR_IAMAGES = '/kaggle/working/Road-Lines-Segmentation-10/train/images'
VAL_DIR_IAMGES = '/kaggle/working/Road-Lines-Segmentation-10/valid/images'
TEST_DIR_IMAGES = '/kaggle/working/Road-Lines-Segmentation-10/test/images'

In [30]:
print(f"Total Training: {len(os.listdir(TRAIN_DIR_IAMAGES))} images")
print(f"Total Validation: {len(os.listdir(VAL_DIR_IAMGES))} images")
print(f"Total Testing: {len(os.listdir(TEST_DIR_IMAGES))} images")

Total Training: 15632 images
Total Validation: 1467 images
Total Testing: 800 images


## check if the GPU is avalabie

In [20]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 12 17:42:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## upload the model

In [6]:
#model = YOLO('/kaggle/input/best/pytorch/default/1/best.pt')
# first experiment using the pervious model from the mini project

In [33]:
# second experiment usning yolov8n trained on coco dataset
model = YOLO('yolov8n-seg.yaml')
model = YOLO('yolov8n-seg.pt')
model = YOLO('yolov8n-seg.yaml').load('yolov8n.pt')

100%|██████████| 6.74M/6.74M [00:00<00:00, 77.8MB/s]


100%|██████████| 6.25M/6.25M [00:00<00:00, 72.6MB/s]


Transferred 355/417 items from pretrained weights


## overwrite the data.yaml file

In [34]:
%%writefile /kaggle/working/Road-Lines-Segmentation-10/data.yaml
names:
- solid-yellow-line
- car
- truck
- bus
nc: 4
roboflow:
  license: CC BY 4.0
  project: road-lines-segmentation-m3thn
  url: https://universe.roboflow.com/first-fwyb3/road-lines-segmentation-m3thn/dataset/10
  version: 10
  workspace: first-fwyb3
test: /kaggle/working/Road-Lines-Segmentation-10/test/images
train: /kaggle/working/Road-Lines-Segmentation-10/train/images
val: /kaggle/working/Road-Lines-Segmentation-10/valid/images


Overwriting /kaggle/working/Road-Lines-Segmentation-10/data.yaml


## Hyperparameters, finetuning

In [29]:
IMG_SIZE = 640
BATCH_SIZE = 32
EPOCHS = 50

In [35]:
results = model.train(
    data = '/kaggle/working/Road-Lines-Segmentation-10/data.yaml',
    task = 'segment',
    model = model,
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Model-segment',
    project = 'experiment4',
    patience = 10,
    save = True,
    optimizer = 'auto'
)

Ultralytics YOLOv8.2.92 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]
2024-09-12 18:23:51,456	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-12 18:23:52,104	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all       1467       1466      0.914      0.871      0.914      0.807      0.921      0.877      0.918      0.641
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 20, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 2.146 hours.
Optimizer stripped from experiment4/Model-segment/weights/last.pt, 6.8MB
Optimizer stripped from experiment4/Model-segment/weights/best.pt, 6.8MB

Validating experiment4/Model-segment/weights/best.pt...
Ultralytics YOLOv8.2.92 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,844 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:16<00:00,  1.42it/s]


                   all       1467       1466      0.927      0.874       0.92      0.815      0.926      0.882      0.929      0.637
     solid-yellow-line       1410       1466      0.927      0.874       0.92      0.815      0.926      0.882      0.929      0.637
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to experiment4/Model-segment


lr/pg0,▁▅███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂
lr/pg1,▁▅███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂
lr/pg2,▁▅███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂
metrics/mAP50(B),▅▂▃▁▅▅▇▄▄▇▆▆▇▅▆▅▄▅▃█▅▅▅▆▇▆▆▇▅█
metrics/mAP50(M),▃▁▁▁▄▄▅▂▃▆▄▅▆▃▅▄▃▅▃█▄▄▄▆▆▅▅▇▅█
metrics/mAP50-95(B),▂▃▁▂▅▄▆▃▅▇▆▅█▆▇▆▆▆▅█▇▇▆▆▇▆▇▇▇█
metrics/mAP50-95(M),▃▁▁▂▂▂▄▂▃▇▆▄▇▆▇▅▅▇▆▇▇▇▇▇█▇▇▇▇▇
metrics/precision(B),▅▁▆▃▅▃▅▇▃▅▆▄▂█▄▃▆▄▄▆▆▅▅▆▄▄▄▄▅▆
metrics/precision(M),▇▁▆▄▄▂▃▇▄▄▆▅▂█▅▄▄▄▄▆▆▅▅▅▅▅▄▄▅▆
metrics/recall(B),▂█▁▆▆▆▃▃▅▆▃██▅▇▇▆▇▅▆▄▆▇▃▆▇▅▆▂▆
metrics/recall(M),▂█▁▆▆▇▄▂▄█▂▇█▄█▇▇▇▅▆▅▆▇▄▆▇▅█▄▆
